In [37]:
import finnhub
import pandas as pd
import numpy as np
import plotly.express as px

finnhub_client = finnhub.Client(api_key="d4nj2vhr01qk2nuc6b20d4nj2vhr01qk2nuc6b2g")
def form4_pull(tckr):
    data = finnhub_client.stock_insider_transactions(f"{tckr}", '2014-10-01', '2025-12-01')
    return data

In [38]:
tckr = "AIR"
form_4s = form4_pull(tckr)
form_4s

{'data': [{'change': 436,
   'currency': '',
   'filingDate': '2025-12-01',
   'id': '0000001750-25-000005',
   'isDerivative': True,
   'name': 'WALFISH MARC JAY',
   'share': 53442,
   'source': 'sec',
   'symbol': 'AIR',
   'transactionCode': 'A',
   'transactionDate': '2025-11-28',
   'transactionPrice': 83.21},
  {'change': 172,
   'currency': '',
   'filingDate': '2025-12-01',
   'id': '0000001750-25-000007',
   'isDerivative': True,
   'name': 'LEDUC ROBERT F',
   'share': 5092,
   'source': 'sec',
   'symbol': 'AIR',
   'transactionCode': 'A',
   'transactionDate': '2025-11-28',
   'transactionPrice': 83.21},
  {'change': -38462,
   'currency': '',
   'filingDate': '2025-11-06',
   'id': '0000001750-25-000003',
   'isDerivative': False,
   'name': 'Holmes John McClain III',
   'share': 260141,
   'source': 'sec',
   'symbol': 'AIR',
   'transactionCode': 'S',
   'transactionDate': '2025-11-05',
   'transactionPrice': 83.501},
  {'change': 479,
   'currency': '',
   'filingDate'

In [39]:
df = pd.DataFrame(form_4s["data"]).set_index("name")
fig1 = px.line(df, x = "transactionDate", y = "share", color = df.index)
fig1

In [83]:
adj_df = df[["transactionDate","change","share"]]
adj_df

,transactionDate,change,share
name,,,
WALFISH MARC JAY,2025-11-28,436,53442
LEDUC ROBERT F,2025-11-28,172,5092
Holmes John McClain III,2025-11-05,-38462,260141
WALFISH MARC JAY,2025-08-29,479,53006
LEDUC ROBERT F,2025-08-29,188,4920
...,...,...,...
WALFISH MARC JAY,2015-03-02,408,408
FOGLEMAN RONALD R,2015-03-02,649,649
STORCH DAVID P,2014-12-23,-3000,906815


In [138]:
ins_df = {name: adj_df.loc[name].set_index("transactionDate") for name in adj_df.index.unique()}
ins_df

{'WALFISH MARC JAY':                  change   share
 transactionDate                
 2025-11-28          436   53442
 2025-08-29          479   53006
 2025-06-01         2198  131740
 2025-05-30          590   52527
 2025-02-28          558   51937
 2024-11-29          521   51379
 2024-08-30          551   50858
 2024-06-01         1901  129542
 2024-05-31          511   50307
 2024-02-29          543   49796
 2023-11-30          523   49253
 2023-08-31          832   48730
 2023-06-01         2426  127641
 2023-05-31          623   47898
 2023-02-28          575   47275
 2022-11-30          510   46700
 2022-08-31         1020    1020
 2022-06-01         2574  125215
 2022-05-31          493     493
 2022-02-28          528     528
 2021-11-30          727     727
 2021-08-31         1057    1057
 2021-06-01          293     293
 2021-06-01         2937  122641
 2021-03-01          301     301
 2020-11-30          352     352
 2020-08-31          646     646
 2020-06-01          56

In [139]:
pop_names = []
for name, df_in in ins_df.items():

        if df_in.shape[0] < 7:
            pop_names.append(name)

for name in pop_names:
    del ins_df[name]

In [140]:
def log_df_func(df_dict: dict):
    epsilon = 10**-9
    column = ["change","share"]
    log_ins_group = {name_in: df_in.copy() for name_in, df_in in df_dict.items()}

    for name, df_in in log_ins_group.items():

        for col in column:
            df_in[col] = df_in[col].astype("float64")

            smallest = df_in[col].loc[df_in[col] != 0].min()
            largest = df_in[col].loc[df_in[col] != 0].max()
            if pd.isna(largest):
                continue
            ratio = max(abs(largest),abs(smallest))/min(abs(largest),abs(smallest))

            if ratio > 10:
                pos_mask = df_in[col] > 0
                neg_mask = df_in[col] < 0

                df_in.loc[pos_mask, col] = np.log(df_in[col].loc[pos_mask] + epsilon)
                df_in.loc[neg_mask, col] = -1 * np.log(abs(df_in[col].loc[neg_mask]) + epsilon)
    return log_ins_group
log_dict = log_df_func(ins_df)
log_dict


{'WALFISH MARC JAY':                    change      share
 transactionDate                     
 2025-11-28       6.077642  10.886352
 2025-08-29       6.171701  10.878160
 2025-06-01       7.695303  11.788586
 2025-05-30       6.380123  10.869083
 2025-02-28       6.324359  10.857787
 2024-11-29       6.255750  10.846985
 2024-08-30       6.311735  10.836793
 2024-06-01       7.550135  11.771760
 2024-05-31       6.236370  10.825900
 2024-02-29       6.297109  10.815690
 2023-11-30       6.259581  10.804726
 2023-08-31       6.723832  10.794050
 2023-06-01       7.793999  11.756977
 2023-05-31       6.434547  10.776829
 2023-02-28       6.354370  10.763737
 2022-11-30       6.234411  10.751499
 2022-08-31       6.927558   6.927558
 2022-06-01       7.853216  11.737788
 2022-05-31       6.200509   6.200509
 2022-02-28       6.269096   6.269096
 2021-11-30       6.588926   6.588926
 2021-08-31       6.963190   6.963190
 2021-06-01       5.680173   5.680173
 2021-06-01       7.985144  11

In [141]:
name_list = [name for name in log_dict.keys()]

In [143]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


def db_scan_func(name_in):
    df_in = log_dict[name_in]

    X_in = df_in
    X_scaled_in = StandardScaler().fit_transform(X_in)


    db = DBSCAN(eps = 0.5, min_samples = 5)
    labels = db.fit_predict(X_scaled_in)

    df_in["cluster"] = labels.astype(str)
    df_in["is_anomaly"] = (labels == -1).astype(int)

#######
    # 2) Get all non-anomaly clusters as strings
    unique_clusters = sorted(c for c in df_in["cluster"].unique() if c != "-1")

    # 3) Take a list of blues (slice, not single item!)
    base_blues = px.colors.sequential.Blues  # this is a list like ['#f7fbff', ..., '#08306b']

    # pick as many as we need
    blue_palette = base_blues[:len(unique_clusters)]  # or base_blues[-len(unique_clusters):]

    # 4) Build the color map
    color_map = {"-1": "red"}  # anomaly color
    for c, color in zip(unique_clusters, blue_palette):
        color_map[c] = color
#######


    fig2_in = px.scatter(
        df_in,
        x = "change",
        y = "share",
        color = "cluster",
        color_discrete_map = color_map,
        hover_name = df_in.index,
        title = f"{name_in} - DBSCAN"
    )

    return df_in, fig2_in, X_scaled_in

name = name_list[0]
ins_df, fig2, X_scaled = db_scan_func(name)

In [144]:
fig2

In [145]:
ins_df

,change,share,cluster,is_anomaly
transactionDate,,,,
2025-11-28,6.077642,10.886352,0,0
2025-08-29,6.171701,10.878160,0,0
2025-06-01,7.695303,11.788586,1,0
2025-05-30,6.380123,10.869083,0,0
2025-02-28,6.324359,10.857787,0,0
2024-11-29,6.255750,10.846985,0,0
2024-08-30,6.311735,10.836793,0,0
2024-06-01,7.550135,11.771760,1,0
2024-05-31,6.236370,10.825900,0,0
